
<a id='types-methods'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# A Necessidade de Velocidade

## Conteúdo

- [A Necessidade de Velocidade](#A-Necessidade-de-Velocidade)  
  - [Resumo](#Resumo)  
  - [Entendendo Despacho Múltiplo em Julia](#Entendendo-Despacho-Múltiplo-em-Julia)  
  - [Fundações](#Fundações)  
  - [Compilação JIT em Julia](#Compilação-JIT-em-Julia)  
  - [Código Rápido e Devagar em Julia](#Código-Rápido-e-Devagar-em-Julia)  
  - [Mais Comentários](#Mais-Comentários)  

> *Devidamente traduzido, revisado e adaptado do [QuantEcon](https://quantecon.org/) pelos bolsistas CNPq, Pedro Luiz H. Furtado e Jonas Aragão M. Corpes, sob supervisão do Prof. Christiano Penna, do CAEN/UFC.*

## Resumo

Cientistas da Computação geralmente classificam linguagens de programação de acordo com as seguintes duas categorias:

*Linguagens de alto nível* com o objetivo de maximizar a produtivade por:

- Sendo fáceis de ler, escrever e limpar. 
- Automatização de tarefas padrão (por exemplo, gerenciamento de memória).  
- sendo interativas, etc.  


*Linguagens de baixo nível* com o objetivo para velocidade e controle, que elas alcançam por:

- Ser mais próximo das máquinas (acesso direto a CPU, memória, etc.).
- Requere relativamente uma grande quantidade de informação do usuário (por exemplo, todos os tipos de dados devem ser especificados).  


Tradicionalmente nos entendemos isso como um dilema:

- Alta produtuvidade ou alta performance. 
- Otimizado para humanos ou otimizados para máquinas.  


Uma das grandes forças do Julia é que ele empurra a curva, alcançando ambas alta produtividade e alta performance como relativamente pouco ruído.

A palavra "relativamente" é importante aqui, no entanto…

Em programas simples, para alcançar performance excelente é geralmente trivial.

Para mais longos, programas mais sofisticados, você deve está atento sobre potênciais obstáculo.

Essa aula cobre esses pontos chaves.

### Requerimentos

Você deve ler nossa [aula anterior](https://julia.quantecon.org/more_julia/generic_programming.html) em tipos, métodos e dispacho simultâneo antes dessa aula.

###  Configuração

In [2]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0", instantiate = true)

Activated C:\Users\cliente\Downloads\Project.toml
Info quantecon-notebooks-julia 0.4.0 activated, 0.5.0 requested


In [18]:
using LinearAlgebra, Statistics

## Entendendo Despacho Múltiplo em Julia

Esta seção fornece mais base em como métodos, funções e tipos estão conectados:

### Métodos e Funções

O tipo preciso dos dados é importante, por ambas as razões eficiência e correção matemática.

Por exemplo considere 1 + 1 vs. 1.0 + 1.0 ou [1 0] + [0 1].

Em uma CPU, a adição de um interio e ponto flutuante são coisas diferentes, usando um conjunto diferente de instruções.

Julia trata esse problema armazenando múltiplos, versões especializadas de funções como adição, uma para cada tipo de dado ou conjuntos de tipos de dados.

Essas versões especializadas individuais são chamadas de **métodos**.

Quando uma operação como a adição é requerida, o compilador do Julia inspeciona os tipos de dados para agir e entregar o método apropriado.

Este processo é chamado de **despacho múltiplo**.

Como todos operados “infix” 1 + 1 tem uma sintaxe alternativa +(1, 1).

In [8]:
+(1, 1)

2

Este operador + é uma função de vários métodos.

Podemos investigar eles usando o macro @which que mostra o método para o qual uma determinada chamada é despachada:

In [9]:
x, y = 1.0, 1.0
@which +(x, y)

+(x::Float64, y::Float64) in Base at float.jl:401

Vemos que a operação é enviada para o método especializado `+`  em adicionar números de ponto flutuante.

Aqui está o caso inteiro:

In [10]:
x, y = 1, 1
@which +(x, y)

+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:53

Esta saída diz que a chamada foi despachada para o método + responsável pelo tratamento de valores inteiros.

Saiba mais sobre os detalhes dessa sintaxe abaixo.

Aqui está outro exemplo, com números complexos.

In [11]:
x, y = 1.0 + 1.0im, 1.0 + 1.0im
@which +(x, y)

+(z::Complex, w::Complex) in Base at complex.jl:271

Novamente, a chamada foi enviada para um método + projetado especificamente para manipular o tipo de dados fornecido.

#### Adicionando Métodos

É simples adicionar métodos às funções existentes.

Por exemplo, no momento não podemos adicionar um número inteiro e uma string em Julia (ou seja `100 + "100"` , não é uma sintaxe válida).

Esse é um comportamento sensato, mas se você quiser mudar, não há nada para impedi-lo.

In [12]:
import Base: +  # permite adicionar métodos a função +

+(x::Integer, y::String) = x + parse(Int, y)

@show +(100, "100")
@show 100 + "100";  # equivalente

100 + "100" = 200
100 + "100" = 200


### Entendendo o Processo de Compilação 

Agora podemos ser um pouco mais claros sobre o que acontece quando você chama uma função em determinados tipos.

Suponha que executemos a chamada de função `f(a, b)` onde `a` e `b`
são tipos concretos `S` e `T` respectivamente.

O interpretador do Julia primeiro consulta os tipos de `a` e `b` para obter o *tuple* `(S, T)`.

Em seguida, analisa a lista de métodos pertencentes a `f`, procurando uma correspondência.

Se encontrar um método correspondente `(S, T)`, chama esse método.

Caso contrário, ele verifica se o par `(S, T)` corresponde a algum método definido para os tipos *parentes imediato*.

Por exemplo, se `S` é `Float64` e `T` é `ComplexF32` então os 
parentes imediatos são `AbstractFloat` e `Number` respectivamente.

In [13]:
supertype(Float64)

AbstractFloat

In [14]:
supertype(ComplexF32)

Number

Portanto, o intérprete procura a seguir um método do formulário`f(x::AbstractFloat, y::Number)`.

Se o intérprete não consegue encontrar uma correspondência nos *parentes imediatos* (supertipos), ele prossegue na árvore, observando os *parentes* do último tipo verificado em cada iteração.

- Se, eventualmente, encontrar um método correspondente, ele invocará esse método.
- Caso contrário, obtemos um erro.  


Este é o processo que leva ao seguinte erro (já que adicionamos apenas o `+`para adicionar `Integer`e `String` acima).

In [16]:
@show (typeof(100.0) <: Integer) == false
100.0 + "100"

(typeof(100.0) <: Integer) == false = true


MethodError: MethodError: no method matching +(::Float64, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(::Float64, !Matched::Float64) at float.jl:401
  +(::AbstractFloat, !Matched::Bool) at bool.jl:106
  ...

Como o procedimento de despacho começa a partir de tipos concretos e trabalha para cima, o despacho sempre invoca o método mais específico disponível.

Por exemplo, se você tiver métodos para a função`f` que manipula:

1. `(Float64, Int64)` pares  
1. `(Number, Number)` pares  


e você chama `f` com `f(0.5, 1)` em seguida, o primeiro método será invocado.

Isso faz sentido (espero) porque o primeiro método seja otimizado para exatamente esse tipo de dados.

O segundo método é provavelmente mais um método de "pegar tudo" que lida com outros dados de uma maneira menos ideal.

Aqui está outro exemplo simples, envolvendo uma função definida pelo usuário.

In [19]:
function q(x)  # ou q(x::Any)
    println("Default (Any) method invoked")
end

function q(x::Number)
    println("Number method invoked")
end

function q(x::Integer)
    println("Integer method invoked")
end

q (generic function with 3 methods)

Vamos agora executar isso e ver como isso se relaciona com a nossa discussão sobre o envio de métodos acima:

In [20]:
q(3)

Integer method invoked


In [21]:
q(3.0)

Number method invoked


In [22]:
q("foo")

Default (Any) method invoked


Desde então `typeof(3) <: Int64 <: Integer <: Number`, a chamada `q(3)` ´prossegue na árvore `Integer` e invoca `q(x::Integer)`.

Por outro lado, `3.0` é um `Float64`, que não é um subtipo de `Integer`.

Portanto, a chamada `q(3.0)` continua até `q(x::Number)`.

Finalmente, `q("foo")` é tratado pela função que está operando `Any`, uma vez que o `String` não é um subtipo de `Number` ou `Integer`.

### Analisando Função de Retorno de Tipos

Na maioria das vezes, o tempo gasto para "otimizar" o código Julia para executar mais rapidamente é garantir que o compilador possa deduzir corretamente os tipos para todas as funções.

O macro `@code_warntype` nos dar uma dica.

In [23]:
x = [1, 2, 3]
f(x) = 2x
@code_warntype f(x)

Variables
  #self#::Core.Compiler.Const(f, false)
  x::Array{Int64,1}

Body::Array{Int64,1}
1 ─ %1 = (2 * x)::Array{Int64,1}
└──      return %1


O macro `@code_warntype` compila `f(x)` usando o tipo de `x` como um exemplo – isto é, `[1, 2, 3]` é usada como um protótipo para analisar a compilação, em vez de simplesmente calcular o valor.

Aqui o, comando `Body::Array{Int64,1}` nos diz tipo que do valor de retorno da função, quando chamado com tipos como `[1, 2, 3]`, é sempre um vetor de números inteiros.

Em contraste, considere uma função potencialmente retornando `nothing`, como [nesta aula](https://julia.quantecon.org/getting_started_julia/fundamental_types.html).

In [24]:
f(x) = x > 0.0 ? x : nothing
@code_warntype f(1)

Variables
  #self#::Core.Compiler.Const(f, false)
  x::Int64

Body::Union{Nothing, Int64}
1 ─ %1 = (x > 0.0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return Main.nothing


Isso indica que o compilador determina o tipo de retorno quando chamado com um número inteiro (como `1`) poderia ser um dos dois tipos diferentes, `Body::Union{Nothing, Int64}`.

Um exemplo final é uma variação acima, que retorna o máximo de  `x` e `0`.

In [25]:
f(x) = x > 0.0 ? x : 0.0
@code_warntype f(1)

Variables
  #self#::Core.Compiler.Const(f, false)
  x::Int64

Body::Union{Float64, Int64}
1 ─ %1 = (x > 0.0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0.0


O que mostra que, quando chamado com um número inteiro, o tipo pode ser esse número inteiro ou o ponto flutuante `0.0`.

Por outro lado, se usarmos alterar a função para retornar  `0` se x <= 0, é instável com ponto flutuante.

In [26]:
f(x) = x > 0.0 ? x : 0
@code_warntype f(1.0)

Variables
  #self#::Core.Compiler.Const(f, false)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x > 0.0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


A solução é usar a função `zero(x)` que retorna o elemento de identidade aditivo do tipo `x`.

Por outro lado, se mudar a função para retornar  `0` se `x <= 0`, é tipo instável, com ponto flutuante.

In [27]:
@show zero(2.3)
@show zero(4)
@show zero(2.0 + 3im)

f(x) = x > 0.0 ? x : zero(x)
@code_warntype f(1.0)

zero(2.3) = 0.0
zero(4) = 0
zero(2.0 + 3im) = 0.0 + 0.0im
Variables
  #self#::Core.Compiler.Const(f, false)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0.0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Main.zero(x)::Core.Compiler.Const(0.0, false)
└──      return %4


## Fundações

Vamos pensar na rapidez com que o código é executado, considerando:

- configuração de hardware.
- algoritmo (isto é, conjunto de instruções a serem executadas). 


Começaremos discutindo os tipos de instruções que as máquinas entendem.

### Código de Máquina

Todas as instruções para computadores acabam como *código de máquina*.

Escrever código rápido - expressando um determinado algoritmo para que ele seja executado rapidamente - se resume a produzir código de máquina eficiente.

Você pode fazer isso sozinho, à mão, se quiser.

Normalmente, isso é feito escrevendo [assembly](https://en.wikipedia.org/wiki/Assembly_language), que é uma representação simbólica do código da máquina.

Aqui está um código de montagem implementando uma função que recebe os argumentos  $ a, b $ e retorna $ 2a + 8b $

```asm
    pushq   %rbp
    movq    %rsp, %rbp
    addq    %rdi, %rdi
    leaq    (%rdi,%rsi,8), %rax
    popq    %rbp
    retq
    nopl    (%rax)
```


Observe que esse código é específico para uma peça de hardware específica que usamos - máquinas diferentes requerem código de máquina diferente.

Se você se sentir tentado a começar a reescrever seu modelo econômico em montagem, contenha-se.

É muito mais sensato dar essas instruções em uma linguagem como Julia, onde elas podem ser facilmente escritas e entendidas.

In [28]:
function f(a, b)
    y = 2a + 8b
    return y
end

f (generic function with 2 methods)

ou Python


```python
def f(a, b):
    y = 2 * a + 8 * b
    return y
```


ou até C

```c
int f(int a, int b) {
    int y = 2 * a + 8 * b;
    return y;
}
```


Em qualquer uma dessas linguagens, acabamos com um código que é muito mais fácil para humanos escrever, ler, compartilhar e depurar.

Deixamos para a própria máquina transformar nosso código em código de máquina.

Como exatamente isso acontece?

### Gerando Código de Máquina

O processo para transformar código de alto nível em código de máquina difere entre linguagens.

Vejamos algumas das opções e como elas diferem umas das outras.

#### Linguagens Compiladas ATQ

Linguagens compiladas tradicionais como Fortran, C e C ++ são uma opção razoável para escrever código rápido.

De fato, a referência padrão de desempenho ainda é C ou Fortran bem escrita.

Essas linguagens são compiladas até um código de máquina eficiente, porque os usuários são forçados a fornecer muitos detalhes sobre os tipos de dados e como o código será executado.

O compilador, portanto, possui amplas informações para criar antecipadamente o código de máquina correspondente (AOT) de uma maneira que:

- organiza os dados de maneira ideal na memória e
- implementa operações eficientes conforme necessário para a tarefa em questão.


Ao mesmo tempo, a sintaxe e a semântica de C e Fortran são detalhadas e pesadas quando comparadas a algo como Julia.

Além disso, essas linguagens de baixo nível carecem da interatividade que é tão crucial para o trabalho científico.

#### Linguagens Interpretadas

Linguagens interpretadas como Python geram código de máquina "on the fly", durante a execução do programa.

Isso permite que eles sejam flexíveis e interativos.

Além disso, os programadores podem deixar muitos detalhes tediosos para o ambiente de tempo de execução, como:

- especificando tipos de variáveis. 
- alocação/desalocação de memória, etc. 


Por exemplo, considere o que acontece quando o Python adiciona uma longa lista de números.

Normalmente, o ambiente de tempo de execução precisa verificar o tipo desses objetos um por um antes de descobrir como adicioná-los.

Isso envolve despesas gerais substanciais.

Também há custos indiretos significativos associados ao acesso aos próprios valores dos dados, que podem não ser armazenados contiguamente na memória.

O código de máquina resultante geralmente é complexo e lento.

#### Compilação Just-in-Time

A compilação Just-in-time (JIT) é uma abordagem alternativa que combina algumas das vantagens da compilação AOT e das linguagens interpretadas.

A idéia básica é que funções para tarefas específicas sejam compiladas conforme solicitado.

Desde que o compilador tenha informações suficientes sobre o que a função faz, ele pode, em princípio, gerar código de máquina eficiente.

Em alguns casos, todas as informações são fornecidas pelo programador.

Em outros casos, o compilador tentará inferir informações ausentes rapidamente, com base no uso.

Com essa abordagem, os ambientes de computação criados em torno dos compiladores JIT visam:

- fornecer todos os benefícios das linguagens de alto nível discutidos acima e, ao mesmo tempo,  
- produzir conjuntos de instruções eficientes quando as funções são compiladas no código da máquina.

## Compilação JIT em Julia

A compilação JIT é a abordagem usada por Julia.

Em um cenário ideal, todas as informações necessárias para gerar código de máquina nativo eficiente são fornecidas ou inferidas.

Nesse cenário, Julia estará em pé de igualdade com o código de máquina de linguagens de baixo nível.

### Um Exemplo

Considere a função:

In [29]:
function f(a, b)
    y = (a + 8b)^2
    return 7y
end

f (generic function with 2 methods)

Suponha que chamamos `f` com argumentos inteiros (por exemplo, `z = f(1, 2)`).

O compilador JIT conhece os tipos de `a` e `b`.

Além disso, pode inferir tipos para outras variáveis dentro da função.

- por exemplo, `y` também será um número inteiro.

Em seguida, compila uma versão especializada da função para manipular números inteiros e armazená-la na memória.

Podemos visualizar o código de máquina correspondente usando o macro `@code_native`

In [30]:
@code_native f(1, 2)

	.text
; ┌ @ In[29]:2 within `f'
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ int.jl:53 within `+'
	leaq	(%rcx,%rdx,8), %rcx
; │└
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ int.jl:54 within `*'
	imulq	%rcx, %rcx
; │└└
; │ @ In[29]:3 within `f'
; │┌ @ int.jl:54 within `*'
	leaq	(,%rcx,8), %rax
	subq	%rcx, %rax
; │└
	popq	%rbp
	retq
	nopl	(%rax)
; └


Se agora chamarmos `f` novamente, mas desta vez com argumentos de ponto flutuante, o compilador JIT mais uma vez inferirá tipos para as outras variáveis dentro da função.

- por exemplo, `y` será também flutuante.


Em seguida, ele compila uma nova versão para lidar com esse tipo de argumento.

In [31]:
@code_native f(1.0, 2.0)

	.text
; ┌ @ In[29]:2 within `f'
	pushq	%rbp
	movq	%rsp, %rbp
	movabsq	$370318600, %rax        # imm = 0x16129D08
; │┌ @ promotion.jl:312 within `*' @ float.jl:405
	vmulsd	(%rax), %xmm1, %xmm1
; │└
; │┌ @ float.jl:401 within `+'
	vaddsd	%xmm0, %xmm1, %xmm0
; │└
; │┌ @ intfuncs.jl:244 within `literal_pow'
; ││┌ @ float.jl:405 within `*'
	vmulsd	%xmm0, %xmm0, %xmm0
	movabsq	$370318608, %rax        # imm = 0x16129D10
; │└└
; │ @ In[29]:3 within `f'
; │┌ @ promotion.jl:312 within `*' @ float.jl:405
	vmulsd	(%rax), %xmm0, %xmm0
; │└
	popq	%rbp
	retq
	nopw	(%rax,%rax)
; └


As chamadas subsequentes usando números flutuantes ou números inteiros agora são roteadas para o código compilado apropriado.

### Problemas Potenciais

Em alguns sentidos, o que vimos acima foi o melhor cenário.

Às vezes, o compilador JIT produz código de máquina lento e confuso.

Isso acontece quando a inferência de tipo falha ou o compilador possui informações insuficientes para otimizar efetivamente.

A próxima seção aborda situações em que esses problemas surgem e como contorná-los.

## Código Rápido e Devagar em Julia

Para resumir o que aprendemos até agora, Julia fornece uma plataforma para gerar código de máquina altamente eficiente com relativamente pouco esforço combinando:

1. Compilação JIT.  
1. Declarações de tipo opcionais e inferência de tipo para identificar os tipos de variáveis e, portanto, compilar código eficiente.
1. Envio múltiplo para facilitar a especialização e otimização do código compilado para diferentes tipos de dados. 


Mas o processo não é perfeito e podem ocorrer alguns *gargalos*.

O objetivo desta seção é destacar possíveis problemas e mostrar como contorná-los.

### BenchmarkTools

O principal pacote de Julia para benchmarking é o [BenchmarkTools.jl](https://www.github.com/JuliaCI/BenchmarkTools.jl).

Abaixo, usaremos o macro `@btime` que exporta para avaliar o desempenho do código Julia.

Como mencionado em uma [aula anterior](https://julia.quantecon.org/more_julia/testing.html), também podemos salvar os resultados de benchmark em um arquivo e proteger contra as regressões de desempenho no código.

Para mais, consulte os documentos do pacote.

### Variáveis Globais

Variáveis globais são nomes atribuídos a valores fora de qualquer definição de função ou tipo.

Os programadores são convenientes e iniciantes normalmente os usam com abandono.

Porém, variáveis globais também são perigosas, especialmente em programas de tamanho médio a grande, pois:

- elas podem afetar o que acontece em qualquer parte do seu programa.
- elas podem ser alterados por qualquer função. 


Isso torna muito mais difícil ter certeza sobre o que uma pequena parte de um determinado pedaço de código realmente comanda.

Aqui está uma [discussão útil sobre esse tópicos](http://wiki.c2.com/?GlobalVariablesAreBad).

Quando se trata de compilação JIT, variáveis globais criam problemas adicionais.

O motivo é que o compilador nunca pode ter certeza do tipo da variável global ou mesmo que o tipo permanecerá constante enquanto uma determinada função for executada.

Para ilustrar, considere este código, onde  `b` é global.

In [32]:
b = 1.0
function g(a)
    global b
    for i ∈ 1:1_000_000
        tmp = a + b
    end
end

g (generic function with 1 method)

O código é executado de forma relativamente lenta e usa uma quantidade enorme de memória.

In [33]:
using BenchmarkTools

@btime g(1.0)

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1273


  19.228 ms (2000000 allocations: 30.52 MiB)


Se você olhar para o código de máquina correspondente, verá que está uma bagunça.

In [34]:
@code_native g(1.0)

	.text
; ┌ @ In[32]:3 within `g'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	andq	$-32, %rsp
	subq	$96, %rsp
	vmovaps	%xmm6, -64(%rbp)
	vmovaps	%xmm0, %xmm6
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 32(%rsp)
	movl	$jl_get_ptls_states, %eax
	vzeroupper
	callq	*%rax
	movq	%rax, %rsi
	movq	$4, 32(%rsp)
	movq	(%rsi), %rax
	movq	%rax, 40(%rsp)
	leaq	32(%rsp), %rax
	movq	%rax, (%rsi)
	movl	$1000000, %ebx          # imm = 0xF4240
	movabsq	$jl_gc_pool_alloc, %r14
	movabsq	$jl_apply_generic, %r15
	leaq	64(%rsp), %r12
	nopl	(%rax)
; │ @ In[32]:5 within `g'
L112:
	movq	291904136, %rdi
	movq	%rdi, 48(%rsp)
	movl	$1400, %edx             # imm = 0x578
	movl	$16, %r8d
	movq	%rsi, %rcx
	callq	*%r14
	movq	$jl_system_image_data, -8(%rax)
	vmovsd	%xmm6, (%rax)
	movq	%rax, 56(%rsp)
	movq	%rax, 64(%rsp)
	movq	%rdi, 72(%rsp)
	movl	$jl_system_image_data, %ecx
	movl	$2, %r8d
	movq	%r12, %rdx
	callq	*%r15
; │┌ @ range.jl:597 within `iterate'
; ││┌ @ p

Se eliminarmos a variável global assim:

In [35]:
function g(a, b)
    for i ∈ 1:1_000_000
        tmp = a + b
    end
end

g (generic function with 2 methods)

então a velocidade de execução melhora drasticamente.

In [36]:
@btime g(1.0, 1.0)

  1.299 ns (0 allocations: 0 bytes)


Observe que a segunda execução foi dramaticamente mais rápida que a primeira.

Isso ocorre porque a primeira chamada incluiu o tempo para a compilação do JIT.

Observe também quão pequena é a pegada de memória da execução.

Além disso, o código da máquina é simples e limpo

In [37]:
@code_native g(1.0, 1.0)

	.text
; ┌ @ In[35]:2 within `g'
	pushq	%rbp
	movq	%rsp, %rbp
; │ @ In[35]:3 within `g'
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
; └


Agora, o compilador tem alguns tipos ao longo da execução da função e, portanto, pode otimizar adequadamente.

#### A palavra chave `const` 

Outra maneira de estabilizar o código acima é manter a variável global, mas anexá-la com `const`.

In [38]:
const b_const = 1.0
function g(a)
    global b_const
    for i ∈ 1:1_000_000
        tmp = a + b_const
    end
end

g (generic function with 2 methods)

Agora o compilador pode novamente gerar código de máquina eficiente.

Vamos deixar você experimentar.

### Tipos Compostos  com  Tipos Abstratos de Campos

Outro cenário que desativa o compilador JIT é quando os tipos compostos têm campos com tipos abstratos.

Nós encontramos esse problema [anteriormente](https://julia.quantecon.org/more_julia/generic_programming.html#spec-field-types), quando discutimos os modelos AR(1).

Vamos experimentar, usando, respectivamente,

- um campo não digitado.
- um campo com tipo abstrato e  
- digitação paramétrica.


Como veremos, a última opção oferece o melhor desempenho, mantendo uma flexibilidade significativa.

Aqui está o caso não digitado:

In [39]:
struct Foo_generic
    a
end

Aqui está o caso de um tipo abstrato no campo `a`:

In [40]:
struct Foo_abstract
    a::Real
end

Finalmente, aqui está o caso digitado parametricamente:

In [41]:
struct Foo_concrete{T <: Real}
    a::T
end

Agora geramos instâncias:

In [42]:
fg = Foo_generic(1.0)
fa = Foo_abstract(1.0)
fc = Foo_concrete(1.0)

Foo_concrete{Float64}(1.0)

No último caso, informações concretas do tipo para os campos são incorporadas no objeto.

In [43]:
typeof(fc)

Foo_concrete{Float64}

Isso é significativo porque essas informações são detectadas pelo compilador.

#### Timing

Aqui está uma função que usa o campo `a` de nossos objetos.

In [44]:
function f(foo)
    for i ∈ 1:1_000_000
        tmp = i + foo.a
    end
end

f (generic function with 2 methods)

Vamos tentar cronometrar nosso código, começando com o caso genérico:

In [45]:
@btime f($fg)

  24.651 ms (1999489 allocations: 30.51 MiB)


O momento não é muito impressionante.

Aqui está o código de máquina desagradável:

In [46]:
@code_native f(fg)

	.text
; ┌ @ In[44]:2 within `f'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	subq	$88, %rsp
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%xmm0, -80(%rbp)
	movq	%rdx, %rdi
	movq	$0, -64(%rbp)
	movq	%rdi, -104(%rbp)
	movl	$jl_get_ptls_states, %eax
	callq	*%rax
	movq	%rax, %r14
	movq	$2, -80(%rbp)
	movq	(%r14), %rax
	movq	%rax, -72(%rbp)
	leaq	-80(%rbp), %rax
	movq	%rax, (%r14)
	movq	(%rdi), %rsi
	movl	$1, %edi
	movabsq	$jl_box_int64, %r15
	movabsq	$jl_apply_generic, %r12
	leaq	-96(%rbp), %r13
	nopl	(%rax,%rax)
; │ @ In[44]:3 within `f'
; │┌ @ Base.jl:20 within `getproperty'
L112:
	movq	(%rsi), %rbx
; │└
	movq	%rdi, %rcx
	callq	*%r15
	movq	%rax, -64(%rbp)
	movq	%rax, -96(%rbp)
	movq	%rbx, -88(%rbp)
	movl	$jl_system_image_data, %ecx
	movl	$2, %r8d
	movq	%r13, %rdx
	callq	*%r12
; │┌ @ range.jl:598 within `iterate'
; ││┌ @ int.jl:53 within `+'
	addq	$1, %rdi
; │└└
; │┌ @ promotion.jl:401 within `iterate'
	cmpq	$1000001, %rdi      

O caso abstrato é semelhante:

In [47]:
@btime f($fa)

  23.542 ms (1999489 allocations: 30.51 MiB)


Observe a grande área de cobertura da memória.

O código da máquina também é longo e complexo, embora omitamos detalhes.

Finalmente, vamos olhar para a versão parametricamente digitada:

In [48]:
@btime f($fc)

  1.299 ns (0 allocations: 0 bytes)


Parte desse tempo é a compilação JIT, e mais uma execução nos leva a um ponto.

Aqui está o código de máquina correspondente:

In [49]:
@code_native f(fc)

	.text
; ┌ @ In[44]:2 within `f'
	pushq	%rbp
	movq	%rsp, %rbp
; │ @ In[44]:3 within `f'
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
; └


Muito melhor…

### Contêiners Abstratos

Outra maneira de encontrar problemas é com tipos abstratos de contêineres.

Considere a função a seguir, que basicamente faz o mesmo trabalho que a função `sum()` do Julia, mas atua apenas em dados de ponto flutuante.

In [50]:
function sum_float_array(x::AbstractVector{<:Number})
    sum = 0.0
    for i ∈ eachindex(x)
        sum += x[i]
    end
    return sum
end

sum_float_array (generic function with 1 method)

As chamadas para esta função são executadas muito rapidamente.

In [51]:
x = range(0,  1, length = Int(1e6))
x = collect(x)
typeof(x)

Array{Float64,1}

In [52]:
@btime sum_float_array($x)

  847.001 μs (0 allocations: 0 bytes)


499999.9999999796

Quando Julia compila essa função, sabe que os dados passados como `x` serão uma matriz de flutuadores de 64 bits.

Portanto, é sabido pelo compilador que o método relevante `+` é sempre a adição de números de ponto flutuante.

Além disso, os dados podem ser organizados em blocos contínuos de memória de 64 bits para simplificar o acesso à memória.

Por fim, os tipos de dados são estáveis - por exemplo, a variável local `sum` começa como uma flutuação e permanece flutuante por toda parte.

#### Inferências de tipo 

Aqui está a mesma função menos a anotação de tipo na assinatura da função.

In [53]:
function sum_array(x)
    sum = 0.0
    for i ∈ eachindex(x)
        sum += x[i]
    end
    return sum
end

sum_array (generic function with 1 method)

Quando o executamos com a mesma matriz de números de ponto flutuante, ele é executado em uma velocidade semelhante à da função com informações de tipo.

In [54]:
@btime sum_array($x)

  847.400 μs (0 allocations: 0 bytes)


499999.9999999796

O motivo é que, quando `sum_array()` é chamado pela primeira vez em um vetor de um determinado tipo de dados, uma versão recém-compilada da função é produzida para manipular esse tipo.

Nesse caso, como estamos chamando a função em um vetor de flutuantes, obtemos uma versão compilada da função com essencialmente a mesma representação interna que `sum_float_array()`.

#### Um Contêiner Abstrato

As coisas ficam mais difíceis para o intérprete quando o tipo de dados na matriz é impreciso.

Por exemplo, o seguinte snippet cria uma matriz em que o tipo de elemento é `Any`.

In [55]:
x = Any[ 1/i for i ∈ 1:1e6 ];

In [56]:
eltype(x)

Any

Agora, a soma é muito mais lenta e o gerenciamento de memória é menos eficiente.

In [57]:
@btime sum_array($x)

  17.316 ms (1000000 allocations: 15.26 MiB)


14.392726722864989

## Mais Comentários

Aqui estão alguns comentários finais sobre desempenho.

### Digitação Explícita

Escrever código Julia rápido equivale a escrever Julia a partir do qual o compilador pode gerar código de máquina eficiente.

Para isso, Julia precisa saber sobre o tipo de dados que está processando o mais cedo possível.

Poderíamos codificar o tipo de todas as variáveis e argumentos de função, mas isso tem um custo.

Nosso código se torna mais complicado e menos genérico.

Estamos começando a perder as vantagens que nos atraíram para Julia em primeiro lugar.

Além disso, digitar explicitamente tudo não é necessário para o desempenho ideal.

O compilador Julia é inteligente e geralmente pode inferir tipos perfeitamente, sem nenhum custo de desempenho.

O que realmente queremos fazer é:

- manter nosso código simples, elegante e genérico.
- ajudar o compilador em situações nas quais ele é passível de disparar. 

### Resumo e Dicas

Use funções para segregar operações em blocos logicamente distintos.

Os tipos de dados serão determinados nos limites da função.

Se os tipos não forem fornecidos, eles serão inferidos.

Se os tipos forem estáveis e puderem ser deduzidos efetivamente, suas funções serão executadas rapidamente.

### Leitura Adicional

Uma boa próxima parada para leitura adicional é a [parte relevante](https://docs.julialang.org/en/v1/manual/performance-tips/) da documentação do Julia.